# returns

In [1]:
import vectorbt as vbt

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from numba import njit, f8, i8, b1, optional
import empyrical

/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/empyrical/utils.py:32: UserWarning: Unable to import pandas_datareader. Suppressing import error and continuing. All data reading functionality will raise errors; but has been deprecated and will be removed in a later version.
  warnings.warn(msg)


In [7]:
# Disable caching for performance testing
vbt.defaults.caching = False

In [3]:
index = pd.DatetimeIndex([
    datetime(2018, 1, 1),
    datetime(2018, 1, 2),
    datetime(2018, 1, 3),
    datetime(2018, 1, 4),
    datetime(2018, 1, 5)
], freq='D')
columns = ['a', 'b', 'c']
ts = pd.DataFrame({
    'a': [1, 2, 3, 4, 5], 
    'b': [5, 4, 3, 2, 1],
    'c': [1, 2, 3, 2, 1]
}, index=index).astype(np.float32)

print(ts)

              a    b    c
2018-01-01  1.0  5.0  1.0
2018-01-02  2.0  4.0  2.0
2018-01-03  3.0  3.0  3.0
2018-01-04  4.0  2.0  2.0
2018-01-05  5.0  1.0  1.0


In [4]:
big_ts = pd.DataFrame(np.random.randint(10, size=(1000, 1000)).astype(float))
big_ts.index = [datetime(2018, 1, 1) + timedelta(days=i) for i in range(1000)]
big_ts.shape

(1000, 1000)

In [5]:
returns = ts.vbt.pct_change()
print(returns)

big_returns = big_ts.vbt.pct_change()

                   a         b         c
2018-01-01       NaN       NaN       NaN
2018-01-02  1.000000 -0.200000  1.000000
2018-01-03  0.500000 -0.250000  0.500000
2018-01-04  0.333333 -0.333333 -0.333333
2018-01-05  0.250000 -0.500000 -0.500000


In [6]:
# Test year frequency
print(returns.vbt.returns.year_freq)
print(returns['a'].vbt.returns.year_freq)
print(returns.vbt.returns(year_freq='252 days').year_freq)
print(returns['a'].vbt.returns(year_freq='252 days').year_freq)

365 days 00:00:00
365 days 00:00:00
252 days 00:00:00
252 days 00:00:00


In [7]:
print(returns.vbt.returns.ann_factor) # default
print(returns.vbt.returns(year_freq='252 days').ann_factor)

365.0
252.0


In [8]:
print(returns['a'].vbt.returns.daily()) # already daily, do nothing
print(returns.vbt.returns.daily())

%timeit big_returns.vbt.returns.daily()

2018-01-01         NaN
2018-01-02    1.000000
2018-01-03    0.500000
2018-01-04    0.333333
2018-01-05    0.250000
Freq: D, Name: a, dtype: float64
                   a         b         c
2018-01-01       NaN       NaN       NaN
2018-01-02  1.000000 -0.200000  1.000000
2018-01-03  0.500000 -0.250000  0.500000
2018-01-04  0.333333 -0.333333 -0.333333
2018-01-05  0.250000 -0.500000 -0.500000
8.97 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
print(returns['a'].vbt.returns.annual())
print(returns.vbt.returns.annual())

%timeit big_returns.vbt.returns.annual()

2018-01-01    4.0
Freq: 365D, Name: a, dtype: float64
              a    b             c
2018-01-01  4.0 -0.8  2.980232e-08
10.6 ms ± 394 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
print(empyrical.cum_returns(returns['a']))
%timeit empyrical.cum_returns(big_returns[0])

print(returns['a'].vbt.returns.cumulative())
%timeit big_returns[0].vbt.returns.cumulative()

print(returns.vbt.returns.cumulative())
print(returns.vbt.returns.cumulative(start_value=1))
print(returns.vbt.returns.cumulative(start_value=[1, 2, 3])) # also accepts array
%timeit big_returns.vbt.returns.cumulative()

2018-01-01    0.0
2018-01-02    1.0
2018-01-03    2.0
2018-01-04    3.0
2018-01-05    4.0
Freq: D, dtype: float64
1.18 ms ± 48.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2018-01-01    0.0
2018-01-02    1.0
2018-01-03    2.0
2018-01-04    3.0
2018-01-05    4.0
Freq: D, Name: a, dtype: float64
283 µs ± 90.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
              a    b             c
2018-01-01  0.0  0.0  0.000000e+00
2018-01-02  1.0 -0.2  1.000000e+00
2018-01-03  2.0 -0.4  2.000000e+00
2018-01-04  3.0 -0.6  1.000000e+00
2018-01-05  4.0 -0.8  2.980232e-08
              a    b    c
2018-01-01  1.0  1.0  1.0
2018-01-02  2.0  0.8  2.0
2018-01-03  3.0  0.6  3.0
2018-01-04  4.0  0.4  2.0
2018-01-05  5.0  0.2  1.0
              a    b    c
2018-01-01  1.0  2.0  3.0
2018-01-02  2.0  1.6  6.0
2018-01-03  3.0  1.2  9.0
2018-01-04  4.0  0.8  6.0
2018-01-05  5.0  0.4  3.0
14.2 ms ± 393 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
print(empyrical.cum_returns_final(returns['a']))
%timeit empyrical.cum_returns_final(big_returns[0])

print(returns['a'].vbt.returns.total())
%timeit big_returns[0].vbt.returns.total()

print(returns.vbt.returns.total())
%timeit big_returns.vbt.returns.total()

4.000000149011612


/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


314 µs ± 7.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.000000149011612
191 µs ± 89.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    4.000000e+00
b   -8.000000e-01
c    2.980232e-08
dtype: float64
4.53 ms ± 52.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
vbt.defaults.returns['year_freq'] = '252 days' # same as empyrical

In [13]:
print(empyrical.annual_return(returns['a']))
%timeit empyrical.annual_return(big_returns[0])

print(returns['a'].vbt.returns.annualized())
%timeit big_returns[0].vbt.returns.annualized()

print(returns.vbt.returns.annualized())
%timeit big_returns.vbt.returns.annualized()

1.690786886567203e+35


/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


311 µs ± 5.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.6907868865671834e+35
119 µs ± 6.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
a    1.690787e+35
b   -1.000000e+00
c    1.502038e-06
dtype: float64
4.63 ms ± 72.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
print(empyrical.annual_volatility(returns['a'], alpha=3.))
%timeit empyrical.annual_volatility(big_returns[0], alpha=3.)

print(returns['a'].vbt.returns.annualized_volatility(levy_alpha=3.))
%timeit big_returns[0].vbt.returns.annualized_volatility(levy_alpha=3.)

print(returns.vbt.returns.annualized_volatility(levy_alpha=3.))
print(returns.vbt.returns.annualized_volatility(levy_alpha=[1, 2, 3]))
%timeit big_returns.vbt.returns.annualized_volatility(levy_alpha=3.)

2.121838249438074
71.1 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.121838249438074
179 µs ± 73.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    2.121838
b    0.830587
c    4.466341
dtype: float64
a    84.653704
b     2.087463
c     4.466341
dtype: float64
7.26 ms ± 292 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
print(empyrical.calmar_ratio(returns['b']))
%timeit empyrical.calmar_ratio(big_returns[0])

print(returns['b'].vbt.returns.calmar_ratio())
%timeit big_returns[0].vbt.returns.calmar_ratio()

print(returns.vbt.returns.calmar_ratio())
%timeit big_returns.vbt.returns.calmar_ratio()

-1.2500000139698388


/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


513 µs ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
-1.2500000139698388
The slowest run took 4.57 times longer than the fastest. This could mean that an intermediate result is being cached.
212 µs ± 155 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a         NaN
b   -1.250000
c    0.000002
dtype: float64
15.4 ms ± 271 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
print(empyrical.omega_ratio(returns['c'], risk_free=0.01, required_return=0.1))
%timeit empyrical.omega_ratio(big_returns[0], risk_free=0.01, required_return=0.1)

print(returns['c'].vbt.returns.omega_ratio(risk_free=0.01, required_return=0.1))
%timeit big_returns[0].vbt.returns.omega_ratio(risk_free=0.01, required_return=0.1)

print(returns.vbt.returns.omega_ratio(risk_free=0.01, required_return=0.1))
print(returns.vbt.returns.omega_ratio(risk_free=[0.01, 0.02, 0.03], required_return=0.1))
print(returns.vbt.returns.omega_ratio(risk_free=0.01, required_return=[0.1, 0.2, 0.3]))
%timeit big_returns.vbt.returns.omega_ratio(risk_free=0.01, required_return=0.1)

1.7319528661672228
1.72 ms ± 53.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.7319528661672228
195 µs ± 75 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a         inf
b    0.000000
c    1.731953
dtype: float64
a        inf
b    0.00000
c    1.60973
dtype: float64
a         inf
b    0.000000
c    1.727716
dtype: float64
15.9 ms ± 2.89 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
print(empyrical.sharpe_ratio(returns['a'], risk_free=0.01))
%timeit empyrical.sharpe_ratio(big_returns[0], risk_free=0.01)

print(returns['a'].vbt.returns.sharpe_ratio(risk_free=0.01))
%timeit big_returns[0].vbt.returns.sharpe_ratio(risk_free=0.01)

print(returns.vbt.returns.sharpe_ratio(risk_free=0.01))
print(returns.vbt.returns.sharpe_ratio(risk_free=[0.01, 0.02, 0.03]))
%timeit big_returns.vbt.returns.sharpe_ratio(risk_free=0.01)

24.139822936194918
512 µs ± 179 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
24.139822936194918
191 µs ± 98.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    24.139823
b   -39.938439
c     3.517158
dtype: float64
a    24.139823
b   -41.145646
c     3.068159
dtype: float64
8.78 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
print(empyrical.downside_risk(returns['b'], required_return=0.1))
%timeit empyrical.downside_risk(big_returns[0], required_return=0.1)

print(returns['b'].vbt.returns.downside_risk(required_return=0.1))
%timeit big_returns[0].vbt.returns.downside_risk(required_return=0.1)

print(returns.vbt.returns.downside_risk(required_return=0.1))
print(returns.vbt.returns.downside_risk(required_return=[0.1, 0.2, 0.3]))
%timeit big_returns.vbt.returns.downside_risk(required_return=0.1)

6.920801865722236
177 µs ± 53 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.920801865722236
242 µs ± 118 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    0.000000
b    6.920802
c    5.874521
dtype: float64
a    0.000000
b    8.463303
c    8.098765
dtype: float64
9.39 ms ± 228 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
print(empyrical.sortino_ratio(returns['b'], required_return=0.1))
%timeit empyrical.sortino_ratio(big_returns[0], required_return=0.1)

print(returns['b'].vbt.returns.sortino_ratio(required_return=0.1))
%timeit big_returns[0].vbt.returns.sortino_ratio(required_return=0.1)

print(returns.vbt.returns.sortino_ratio(required_return=0.1))
print(returns.vbt.returns.sortino_ratio(required_return=[0.1, 0.2, 0.3]))
%timeit big_returns.vbt.returns.sortino_ratio(required_return=0.1)

-15.32336860018125
415 µs ± 7.86 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
-15.32336860018125
The slowest run took 5.66 times longer than the fastest. This could mean that an intermediate result is being cached.
263 µs ± 230 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a          inf
b   -15.323369
c     2.859808
dtype: float64
a          inf
b   -15.508129
c    -4.148780
dtype: float64
12.4 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
factor_returns = returns['a'] * np.random.uniform(0.8, 1.2, returns.shape[0])
big_factor_returns = big_returns[0] * np.random.uniform(0.8, 1.2, big_returns.shape[0])

In [21]:
print(empyrical.excess_sharpe(returns['a'], factor_returns))
%timeit empyrical.excess_sharpe(big_returns[0], factor_returns)

print(returns['a'].vbt.returns.information_ratio(factor_returns)) # will broadcast
%timeit big_returns[0].vbt.returns.information_ratio(big_factor_returns)

print(returns.vbt.returns.information_ratio(factor_returns))
%timeit big_returns.vbt.returns.information_ratio(big_factor_returns)

-0.5297150189442545
833 µs ± 37.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
-0.5297150189442545
305 µs ± 88.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a   -0.529715
b   -3.042275
c   -1.041734
dtype: float64
9.13 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
print(empyrical.beta(returns['a'], factor_returns))
%timeit empyrical.beta(big_returns[0], factor_returns)

print(returns['a'].vbt.returns.beta(factor_returns))
%timeit big_returns[0].vbt.returns.beta(big_factor_returns)

print(returns.vbt.returns.beta(factor_returns))
%timeit big_returns.vbt.returns.beta(big_factor_returns)

0.8848199490327197
2.4 ms ± 31.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
0.8848199490327197
246 µs ± 70.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    0.884820
b    0.268729
c    1.778061
dtype: float64
12.4 ms ± 228 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
print(empyrical.alpha(returns['a'], factor_returns, risk_free=0.01))
%timeit empyrical.alpha(big_returns[0], factor_returns, risk_free=0.01)

print(returns['a'].vbt.returns.alpha(factor_returns, risk_free=0.01))
%timeit big_returns[0].vbt.returns.alpha(big_factor_returns, risk_free=0.01)

print(returns.vbt.returns.alpha(factor_returns, risk_free=0.01))
print(returns.vbt.returns.alpha(factor_returns, risk_free=[0.01, 0.02, 0.03]))
%timeit big_returns.vbt.returns.alpha(big_factor_returns, risk_free=0.01)

1452.6731735028388
3.3 ms ± 49.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1452.6731735028388
The slowest run took 4.08 times longer than the fastest. This could mean that an intermediate result is being cached.
422 µs ± 301 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    1452.673174
b      -1.000000
c      -1.000000
dtype: float64
a    1452.673174
b      -1.000000
c      -1.000000
dtype: float64
17 ms ± 497 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
print(empyrical.tail_ratio(returns['a']))
%timeit empyrical.tail_ratio(big_returns[0])

print(returns['a'].vbt.returns.tail_ratio())
%timeit big_returns[0].vbt.returns.tail_ratio()

print(returns.vbt.returns.tail_ratio())
%timeit big_returns.vbt.returns.tail_ratio()

3.5238094437960337
390 µs ± 5.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.5238094437960337
141 µs ± 60.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    3.523809
b    0.436842
c    1.947368
dtype: float64
32.6 ms ± 357 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
print(empyrical.value_at_risk(returns.iloc[1:]['a'], cutoff=0.05))
%timeit empyrical.value_at_risk(big_returns[0], cutoff=0.05)

print(returns['a'].vbt.returns.value_at_risk(cutoff=0.05))
%timeit big_returns[0].vbt.returns.value_at_risk(cutoff=0.05)

print(returns.vbt.returns.value_at_risk(cutoff=0.05))
print(returns.vbt.returns.value_at_risk(cutoff=[0.05, 0.06, 0.07]))
%timeit big_returns.vbt.returns.value_at_risk(cutoff=0.05)

0.26250000596046447
220 µs ± 3.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
0.26250000596046447
The slowest run took 4.82 times longer than the fastest. This could mean that an intermediate result is being cached.
199 µs ± 142 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    0.2625
b   -0.4750
c   -0.4750
dtype: float64
a    0.2625
b   -0.4700
c   -0.4650
dtype: float64
19.2 ms ± 649 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
print(empyrical.conditional_value_at_risk(returns.iloc[1:]['a'], cutoff=0.05))
%timeit empyrical.conditional_value_at_risk(big_returns[0], cutoff=0.05)

print(returns['a'].vbt.returns.conditional_value_at_risk(cutoff=0.05))
%timeit big_returns[0].vbt.returns.conditional_value_at_risk(cutoff=0.05)

print(returns.vbt.returns.conditional_value_at_risk(cutoff=0.05))
print(returns.vbt.returns.conditional_value_at_risk(cutoff=[0.05, 0.06, 0.07]))
%timeit big_returns.vbt.returns.conditional_value_at_risk(cutoff=0.05)

0.25
88.9 µs ± 4.71 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
0.25
194 µs ± 94.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    0.25
b   -0.50
c   -0.50
dtype: float64
a    0.25
b   -0.50
c   -0.50
dtype: float64
17.8 ms ± 460 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
print(empyrical.capture(returns['a'], factor_returns))
%timeit empyrical.capture(big_returns[0], big_factor_returns)

print(returns['a'].vbt.returns.capture(factor_returns))
%timeit big_returns[0].vbt.returns.capture(big_factor_returns)

print(returns.vbt.returns.capture(factor_returns))
%timeit big_returns.vbt.returns.capture(big_factor_returns)

0.025115687469450323


/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


584 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
0.025115687469450334
398 µs ± 166 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    2.511569e-02
b   -1.485444e-37
c    2.231193e-43
dtype: float64
7.15 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
print(empyrical.up_capture(returns['a'], factor_returns))
%timeit empyrical.up_capture(big_returns[0], big_factor_returns)

print(returns['a'].vbt.returns.up_capture(factor_returns))
%timeit big_returns[0].vbt.returns.up_capture(big_factor_returns)

print(returns.vbt.returns.up_capture(factor_returns))
%timeit big_returns.vbt.returns.up_capture(big_factor_returns)

0.009998419118704435


/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/empyrical/stats.py:1561: RuntimeWarning: invalid value encountered in double_scalars
  annual_return(factor_returns, period=period))


1.8 ms ± 34.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
0.009998419118704435
329 µs ± 102 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    9.998419e-03
b   -9.221897e-47
c    1.731455e-52
dtype: float64
7.7 ms ± 364 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
print(empyrical.down_capture(returns['a'], factor_returns))
%timeit empyrical.down_capture(big_returns[0], big_factor_returns)

print(returns['a'].vbt.returns.down_capture(factor_returns))
%timeit big_returns[0].vbt.returns.down_capture(big_factor_returns)

print(returns.vbt.returns.down_capture(factor_returns))
%timeit big_returns.vbt.returns.down_capture(big_factor_returns)

nan


/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/empyrical/stats.py:445: RuntimeWarning: invalid value encountered in double_scalars
  return ending_value ** (1 / num_years) - 1


1.9 ms ± 76.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
nan
331 µs ± 77.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a   NaN
b   NaN
c   NaN
dtype: float64
8.24 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
print(returns.vbt.returns.drawdown())

%timeit big_returns.vbt.returns.drawdown()

              a    b         c
2018-01-01  0.0  0.0  0.000000
2018-01-02  0.0 -0.2  0.000000
2018-01-03  0.0 -0.4  0.000000
2018-01-04  0.0 -0.6 -0.333333
2018-01-05  0.0 -0.8 -0.666667
17.9 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
print(empyrical.max_drawdown(returns['b']))
%timeit empyrical.max_drawdown(big_returns[0])

print(returns['b'].vbt.returns.max_drawdown())
%timeit big_returns[0].vbt.returns.max_drawdown()

print(returns.vbt.returns.max_drawdown())
%timeit big_returns.vbt.returns.max_drawdown()

-0.7999999910593033
131 µs ± 3.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
-0.7999999910593032
The slowest run took 4.11 times longer than the fastest. This could mean that an intermediate result is being cached.
178 µs ± 84 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
a    0.000000
b   -0.800000
c   -0.666667
dtype: float64
12.6 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
print(returns.vbt.returns.drawdowns())
%timeit big_returns.vbt.returns.drawdowns()

print(returns.vbt.returns.drawdowns().max_drawdown())

16.8 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
a    0.000000
b   -0.800000
c   -0.666667
dtype: float64
